<a href="https://colab.research.google.com/github/rafaelaugusto1983/Data-Structures-and-Algorithm-in-Python/blob/main/Resumo_de_Normas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import


In [1]:
!pip install -U -q google-generativeai

In [2]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.9 MB/s eta 0:00:00


In [3]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [42]:
from google.colab import userdata

API_KEY=userdata.get('key2')

In [43]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Parse the arguments

In [44]:
model = 'gemini-1.5-pro-latest' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6Ik1lIGRlIG9zIDMgcHJpbmNpcGFpcyBwb250b3MgZGVzc2EgcmVzb2x1w6fDo286IFwiRGlzcMO1ZSBzb2JyZSBvcyBjb25jZWl0b3MgZSBvcyBjcml0w6lyaW9zIGNvbnTDoWJlaXMgYXBsaWPDoXZlaXMgYcKgaW5zdHJ1bWVudG9zIGZpbmFuY2Vpcm9zLCBiZW0gY29tbyBwYXJhIGEgZGVzaWduYcOnw6NvIGUgbyByZWNvbmhlY2ltZW50byBkYXMgcmVsYcOnw7VlcyBkZSBwcm90ZcOnw6NvIChjb250YWJpbGlkYWRlIGRlwqBoZWRnZSkgcGVsYXMgc29jaWVkYWRlcyBjb3JyZXRvcmFzIGRlIHTDrXR1bG9zIGUgdmFsb3JlcyBtb2JpbGnDoXJpb3MsIHBlbGFzIHNvY2llZGFkZXMgZGlzdHJpYnVpZG9yYXMgZGUgdMOtdHVsb3MgZSB2YWxvcmVzIG1vYmlsacOhcmlvcywgcGVsYXMgc29jaWVkYWRlcyBjb3JyZXRvcmFzIGRlIGPDom1iaW8sIHBlbGFzIGFkbWluaXN0cmFkb3JhcyBkZSBjb25zw7NyY2lvIGUgcGVsYXMgaW5zdGl0dWnDp8O1ZXMgZGUgcGFnYW1lbnRvIGF1dG9yaXphZGFzIGEgZnVuY2lvbmFyIHBlbG8gQmFuY28gQ2VudHJhbCBkbyBCcmFzaWwgZSBzb2JyZSBvcyBwcm9jZWRpbWVudG9zIGNvbnTDoWJlaXMgcGFyYSBhIGRlZmluacOnw6NvIGRlIGZsdXhvcyBkZSBjYWl4YXMgZGUgYXRpdm8gZmluYW5jZWlybyBjb21vIHNvbWVudGUgcGFnYW1lbnRvIGRlIHByaW5jaXBhbCBlIGp1cm9zLCBhIGFwbGljYcOnw6NvIGRhIG1ldG9kb2xvZ2lhIHBhcmEgYXB1cmHDp8OjbyBkYSB0YXhhIGRlIGp1cm9zIGVmZXRpdmEgZGUgaW5zdHJ1bWVudG9zIGZpbmFuY2Vpcm9zLCBhIGNvbnN0aXR1acOnw6NvIGRlIHByb3Zpc8OjbyBwYXJhIHBlcmRhcyBhc3NvY2lhZGFzIGFvIHJpc2NvIGRlIGNyw6lkaXRvIGUgYSBldmlkZW5jaWHDp8OjbyBkZSBpbmZvcm1hw6fDtWVzIHJlbGF0aXZhcyBhIGluc3RydW1lbnRvcyBmaW5hbmNlaXJvcyBlbSBub3RhcyBleHBsaWNhdGl2YXMgYSBzZXJlbSBvYnNlcnZhZG9zIHBlbGFzIGluc3RpdHVpw6fDtWVzIGZpbmFuY2VpcmFzIGUgZGVtYWlzIGluc3RpdHVpw6fDtWVzIGF1dG9yaXphZGFzIGEgZnVuY2lvbmFyIHBlbG8gQmFuY28gQ2VudHJhbCBkbyBCcmFzaWwuXCIifSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjoiT3MgMyBwcmluY2lwYWlzIHBvbnRvcyBkYSByZXNvbHXDp8OjbyBzw6NvOlxuXG4xLiAqKkVzdGFiZWxlY2ltZW50byBkZSBjb25jZWl0b3MgZSBjcml0w6lyaW9zIGNvbnTDoWJlaXMqKjogQSByZXNvbHXDp8OjbyBkZWZpbmUgY29tbyBvcyBpbnN0cnVtZW50b3MgZmluYW5jZWlyb3MgZGV2ZW0gc2VyIGNvbnRhYmlsaXphZG9zLCBpbmNsdWluZG8gY29tbyByZWNvbmhlY2VyIGUgZGVzaWduYXIgYXMgcmVsYcOnw7VlcyBkZSBwcm90ZcOnw6NvIChoZWRnZSBhY2NvdW50aW5nKS4gSXNzbyBnYXJhbnRlIG1haW9yIHVuaWZvcm1pZGFkZSBlIHRyYW5zcGFyw6puY2lhIG5hIGNvbnRhYmlsaWRhZGUgZGVzc2FzIGluc3RpdHVpw6fDtWVzLlxuMi4gKipOb3JtYXMgZXNwZWPDrWZpY2FzIHBhcmEgaW5zdGl0dWnDp8O1ZXMgZmluYW5jZWlyYXMqKjogQXMgcmVncmFzIHNlIGFwbGljYW0gZXNwZWNpZmljYW1lbnRlIGEgc29jaWVkYWRlcyBjb3JyZXRvcmFzIGRlIHTDrXR1bG9zIGUgdmFsb3JlcyBtb2JpbGnDoXJpb3MsIGRpc3RyaWJ1aWRvcmFzIGRlIHTDrXR1bG9zIGUgdmFsb3JlcyBtb2JpbGnDoXJpb3MsIGNvcnJldG9yYXMgZGUgY8OibWJpbywgYWRtaW5pc3RyYWRvcmFzIGRlIGNvbnPDs3JjaW8gZSBpbnN0aXR1acOnw7VlcyBkZSBwYWdhbWVudG8uIElzc28gZGVtb25zdHJhIGEgbmVjZXNzaWRhZGUgZGUgdW0gdHJhdGFtZW50byBjb250w6FiaWwgZGlmZXJlbmNpYWRvIHBhcmEgZXNzYXMgaW5zdGl0dWnDp8O1ZXMsIGNvbnNpZGVyYW5kbyBzdWFzIGF0aXZpZGFkZXMgZXNwZWPDrWZpY2FzLlxuMy4gKipQYWRyb25pemHDp8OjbyBkZSBwcm9jZWRpbWVudG9zKio6IEEgcmVzb2x1w6fDo28gZGVmaW5lIHByb2NlZGltZW50b3MgY29udMOhYmVpcyBpbXBvcnRhbnRlcywgY29tbyBhIGRlZmluacOnw6NvIGRlIGZsdXhvcyBkZSBjYWl4YSwgbyBjw6FsY3VsbyBkYSB0YXhhIGRlIGp1cm9zIGVmZXRpdmEsIGEgY29uc3RpdHVpw6fDo28gZGUgcHJvdmlzw6NvIHBhcmEgcGVyZGFzIHBvciByaXNjbyBkZSBjcsOpZGl0byBlIGEgZXZpZGVuY2lhw6fDo28gZGUgaW5mb3JtYcOnw7VlcyBlbSBub3RhcyBleHBsaWNhdGl2YXMuICBBIHBhZHJvbml6YcOnw6NvIGRlc3NlcyBwcm9jZWRpbWVudG9zIGJ1c2NhIGF1bWVudGFyIGEgY29uZmlhYmlsaWRhZGUgZSBjb21wYXJhYmlsaWRhZGUgZGFzIGRlbW9uc3RyYcOnw7VlcyBmaW5hbmNlaXJhcy4ifV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjowLCJtYXhfb3V0cHV0X3Rva2VucyI6ODE5Miwic3RvcF9zZXF1ZW5jZXMiOltdfQ==' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}
user_input_b64 = '' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

In [33]:
contents

[{'role': 'user',
  'parts': 'Me de os 3 principais pontos dessa resolução: "Dispõe sobre os conceitos e os critérios contábeis aplicáveis a\xa0instrumentos financeiros, bem como para a designação e o reconhecimento das relações de proteção (contabilidade de\xa0hedge) pelas sociedades corretoras de títulos e valores mobiliários, pelas sociedades distribuidoras de títulos e valores mobiliários, pelas sociedades corretoras de câmbio, pelas administradoras de consórcio e pelas instituições de pagamento autorizadas a funcionar pelo Banco Central do Brasil e sobre os procedimentos contábeis para a definição de fluxos de caixas de ativo financeiro como somente pagamento de principal e juros, a aplicação da metodologia para apuração da taxa de juros efetiva de instrumentos financeiros, a constituição de provisão para perdas associadas ao risco de crédito e a evidenciação de informações relativas a instrumentos financeiros em notas explicativas a serem observados pelas instituições financeiras

In [34]:
generation_config

{'temperature': 1,
 'top_p': 0.95,
 'top_k': 0,
 'max_output_tokens': 8192,
 'stop_sequences': []}

In [35]:
safety_settings

[{'category': 'HARM_CATEGORY_HARASSMENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_HATE_SPEECH',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'}]

In [36]:
user_input0 = "Me de os 5 pontos principais dessa resolução: "

In [38]:
import requests
import PyPDF2
from io import BytesIO

# URL of the PDF
url = "https://normativos.bcb.gov.br/Lists/Normativos/Attachments/49169/Circ_3598_v3_P.pdf"

# Download the PDF from the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the PDF content
    pdf_content = response.content

    # Create a BytesIO object
    pdf_file = BytesIO(pdf_content)

    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Initialize a variable to store all text
    all_text = ""

    # Extract text from each page
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        all_text += page.extract_text() + "\n"  # Append text from the page to all_text

    # Print or use the all_text variable
    # print(all_text)  # Print all text
    # You can also save the all_text to a file if needed
    # with open("pdf_text.txt", "w", encoding="utf-8") as text_file:
    #     text_file.write(all_text)
else:
    print("Failed to download PDF")

In [46]:
user_input = user_input0 + all_text

### Call the API

In [47]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

chat = gemini.start_chat(history=contents)

response = chat.send_message(
    user_input,
    stream=stream)

In [48]:
display(Markdown(response.text))

## 5 Pontos principais da Circular nº 3.598 de 06/06/2012:

1. **Definição e Espécies de Boleto de Pagamento:** A Circular define o boleto de pagamento como um instrumento padronizado com informações sobre a dívida em cobrança e oferta de produtos/serviços, viabilizando pagamentos e aceite de ofertas. Estabelece três espécies: boleto de cobrança, boleto de proposta e boleto de depósito e aporte.
2. **Agentes Envolvidos:** Define os agentes envolvidos nas operações com boleto: beneficiário (credor), pagador (devedor), instituição financeira recebedora (recebe os fundos) e instituição financeira destinatária (contratada pelo beneficiário para emitir, apresentar e receber os recursos).
3. **Emissão e Apresentação do Boleto:** O boleto deve ser emitido seguindo um modelo preestabelecido, podendo ser apresentado fisicamente ou eletronicamente (mediante concordância prévia do pagador). A Circular determina informações mínimas obrigatórias, como dados do pagador e beneficiário, valor, vencimento, etc.
4. **Liquidação das Obrigações Interbancárias:** A Circular define a sistemática de liquidação das obrigações entre a instituição recebedora e a destinatária. Boletos acima do Valor de Referência (VR-Boleto) são liquidados via STR no mesmo dia, enquanto boletos abaixo do VR-Boleto podem ser liquidados via compensação multilateral ou STR, a critério da recebedora.
5. **Convenção entre Instituições Financeiras:** As instituições financeiras, por meio de suas associações, devem convencionar a padronização do boleto, procedimentos operacionais, horários, direitos e obrigações para garantir uniformidade e cumprimento da legislação. O conteúdo dessa convenção está sujeito à aprovação do Banco Central.

**Observação:** A Circular sofreu alterações posteriores, incorporadas na resposta. É fundamental consultar a versão atualizada para informações precisas.


In [ ]:
response.prompt_feedback

In [ ]:
response.candidates

[content {
  parts {
    text: "Bom dia! \360\237\230\212 Como posso te ajudar hoje? \n"
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]